In [1]:
stock_ids = ['SHIVAMILLS','RSWM', 'VSTIND']

import pandas as pd

#df = pd.read_csv('data/FVT_Scrips.csv')
#stock_ids = df['SCREENER']

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-s:test_list.index('Sep 2021')+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]
    
    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')

        #stock_id = df[df['SCREENER'] == stock_name]['SYMBOL'].values[0]
        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

SHIVAMILLS
14
158
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
202

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,SHIVAMILLS.NS,2020-08-15,23.20,-2.80,9.33,-1.99,131.16,-11.66
7,SHIVAMILLS.NS,2020-11-15,23.50,0.44,36.64,-2.01,135.44,-11.69
8,SHIVAMILLS.NS,2021-02-15,37.40,6.38,45.64,4.14,137.77,9.03
9,SHIVAMILLS.NS,2021-05-16,55.95,5.89,47.34,9.91,138.95,5.65
10,SHIVAMILLS.NS,2021-08-15,90.60,4.47,36.21,17.18,165.83,5.27
11,SHIVAMILLS.NS,2021-11-15,120.20,5.50,53.67,22.24,182.86,5.40
6,RSWM.NS,2020-08-15,71.95,-28.07,216.00,-29.80,2286.00,-2.41
7,RSWM.NS,2020-11-15,90.60,-6.37,554.00,-36.26,2143.00,-2.50
8,RSWM.NS,2021-02-15,170.20,12.98,721.00,-23.65,2103.00,-7.20
9,RSWM.NS,2021-05-16,223.80,30.31,836.00,8.85,2327.00,25.29


In [3]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(TTM)'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS'])*100/tmp['Prev_EPS'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['SG%'] = round((tmp['Sales(TTM)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-3-4e6773986fa5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-3-4e6773986fa5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
<ipython-input-3-4e6773986fa5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
6,SHIVAMILLS.NS,2020-08-15,23.20,-2.80,9.33,-1.99,131.16,-11.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SHIVAMILLS.NS,2020-11-15,23.50,0.44,36.64,-2.01,135.44,-11.69,-1.99,-11.66,23.20,131.16,1.01,0.26,1.27,3.26
8,SHIVAMILLS.NS,2021-02-15,37.40,6.38,45.64,4.14,137.77,9.03,-2.01,-11.69,23.50,135.44,-305.97,365.02,59.05,1.72
9,SHIVAMILLS.NS,2021-05-16,55.95,5.89,47.34,9.91,138.95,5.65,4.14,9.03,37.40,137.77,139.37,-89.56,49.81,0.86
10,SHIVAMILLS.NS,2021-08-15,90.60,4.47,36.21,17.18,165.83,5.27,9.91,5.65,55.95,138.95,73.36,-11.67,61.69,19.35
11,SHIVAMILLS.NS,2021-11-15,120.20,5.50,53.67,22.24,182.86,5.40,17.18,5.27,90.60,165.83,29.45,3.19,32.64,10.27
6,RSWM.NS,2020-08-15,71.95,-28.07,216.00,-29.80,2286.00,-2.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RSWM.NS,2020-11-15,90.60,-6.37,554.00,-36.26,2143.00,-2.50,-29.80,-2.41,71.95,2286.00,21.68,4.54,26.22,-6.26
8,RSWM.NS,2021-02-15,170.20,12.98,721.00,-23.65,2103.00,-7.20,-36.26,-2.50,90.60,2143.00,-34.78,122.69,87.91,-1.87
9,RSWM.NS,2021-05-16,223.80,30.31,836.00,8.85,2327.00,25.29,-23.65,-7.20,170.20,2103.00,-137.42,168.94,31.52,10.65


In [4]:
df_analysis_final[df_analysis_final['Quarter'] == '2021-08-15'].sort_values(by='SG%', ascending=False)

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
10,RSWM.NS,2021-08-15,340.4,15.67,742.00,52.59,2853.00,6.47,8.85,25.29,223.80,2327.00,494.24,-442.24,52.00,22.60
10,SHIVAMILLS.NS,2021-08-15,90.6,4.47,36.21,17.18,165.83,5.27,9.91,5.65,55.95,138.95,73.36,-11.67,61.69,19.35
10,VSTIND.NS,2021-08-15,3383.0,45.62,275.00,197.85,1140.00,17.10,201.26,15.76,3172.75,1111.00,-1.69,8.36,6.67,2.61


In [5]:
df_analysis_final[df_analysis_final['Quarter'] == '2021-11-15'].sort_values(by='SG%', ascending=False)

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
11,RSWM.NS,2021-11-15,462.05,19.00,953.00,77.96,3252.00,5.93,52.59,6.47,340.4,2853.00,48.24,-12.37,35.87,13.99
11,SHIVAMILLS.NS,2021-11-15,120.20,5.50,53.67,22.24,182.86,5.40,17.18,5.27,90.6,165.83,29.45,3.19,32.64,10.27
11,VSTIND.NS,2021-11-15,3325.50,51.73,272.00,192.24,1112.00,17.30,197.85,17.10,3383.0,1140.00,-2.84,1.14,-1.70,-2.46


In [6]:
#df_analysis_final.to_csv('data/FVT_FvS_QoQ_Analysis.csv', index=False)

In [7]:
df_analysis_final[df_analysis_final['Stock'] == 'ANSALBU.BO']

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
